In [1]:
import json
import pandas as pd

file = open(r"D:\BOT_Prime\CIB_DELTA_back\output\data (1).json")
datas = json.load(file)
df = pd.DataFrame(columns=[
        'Name',
        'DOB', 
        'Gender', 
        'FatherName', 
        'CitizenshipDetails', 
        'PassportDetails', 
        'DrivingLicenseDetails',
        'VoterIDDetails',
        'PANDetails', 
        'CompanyDetails',
        'IndianEmbassyDetails',
        'BlackLists' 
    ])
num = 0
for data in datas["BulkOffLoading"]["Institutions"]["Item"]:
    dicts = {}
    data_keys = list(data.keys())
    for data_key in data_keys:
        if isinstance(data[data_key], str):
            dicts[data_key] = data[data_key]
        else:
            # maintain Citizenship details
            keys = list(data[data_key].keys())
            data_count = int(data[data_key][keys[0]])
            if data_count > 1:
                details = data[data_key][keys[1]]
                dicts[data_key] = ['|'.join(entry.values()) for entry in details]
            elif data_count == 0:
                dicts[data_key] = ""
            else:
                details = data[data_key][keys[1]]
                dicts[data_key] = '|'.join(details.values())

    dicts['Account_Nature'] = 'Institutional'
    dicts['Status'] = 'new'
    # num += 1
    # if num == 300:
    #     break

    # appending to df
    df.loc[len(df)] = dicts

In [2]:
# df.tail(20)

In [3]:
new_columns = []
for i, data in df.iterrows():
    # print(type(data))
    if 'PANDetails' in data:
        if isinstance(data['PANDetails'], list):
            for pan_element in data['PANDetails']:
                if isinstance(data['CompanyDetails'],list):
                    for company_element in data['CompanyDetails']:
                        new_data = {
                            'Name': data['Name'],
                            'DOB': data['DOB'],
                            'Gender': data['Gender'],
                            'FatherName': data['FatherName'],
                            'CitizenshipDetails': data['CitizenshipDetails'],
                            'PassportDetails': data['PassportDetails'],
                            'DrivingLicenseDetails': data['DrivingLicenseDetails'],
                            'VoterIDDetails': data['VoterIDDetails'],
                            'PANDetails': pan_element,
                            'CompanyDetails': company_element,
                            'IndianEmbassyDetails': data['IndianEmbassyDetails'],
                            'BlackLists': data['BlackLists']
                        }
                        new_columns.append(new_data)
                elif isinstance(data['CompanyDetails'],str):
                    new_data = {
                            'Name': data['Name'],
                            'DOB': data['DOB'],
                            'Gender': data['Gender'],
                            'FatherName': data['FatherName'],
                            'CitizenshipDetails': data['CitizenshipDetails'],
                            'PassportDetails': data['PassportDetails'],
                            'DrivingLicenseDetails': data['DrivingLicenseDetails'],
                            'VoterIDDetails': data['VoterIDDetails'],
                            'PANDetails': pan_element,
                            'CompanyDetails': data['CompanyDetails'],
                            'IndianEmbassyDetails': data['IndianEmbassyDetails'],
                            'BlackLists': data['BlackLists']
                        }
                    new_columns.append(new_data)
        elif isinstance(data['PANDetails'], str):
            if isinstance(data['CompanyDetails'],str):
                new_data = {
                                'Name': data['Name'],
                                'DOB': data['DOB'],
                                'Gender': data['Gender'],
                                'FatherName': data['FatherName'],
                                'CitizenshipDetails': data['CitizenshipDetails'],
                                'PassportDetails': data['PassportDetails'],
                                'DrivingLicenseDetails': data['DrivingLicenseDetails'],
                                'VoterIDDetails': data['VoterIDDetails'],
                                'PANDetails': data['PANDetails'],
                                'CompanyDetails': data['CompanyDetails'],
                                'IndianEmbassyDetails': data['IndianEmbassyDetails'],
                                'BlackLists': data['BlackLists']
                            }
                new_columns.append(new_data)
            elif isinstance(data['CompanyDetails'],list):
                for company_element in data['CompanyDetails']:
                        new_data = {
                            'Name': data['Name'],
                            'DOB': data['DOB'],
                            'Gender': data['Gender'],
                            'FatherName': data['FatherName'],
                            'CitizenshipDetails': data['CitizenshipDetails'],
                            'PassportDetails': data['PassportDetails'],
                            'DrivingLicenseDetails': data['DrivingLicenseDetails'],
                            'VoterIDDetails': data['VoterIDDetails'],
                            'PANDetails': data['PANDetails'],
                            'CompanyDetails': company_element,
                            'IndianEmbassyDetails': data['IndianEmbassyDetails'],
                            'BlackLists': data['BlackLists']
                        }
                        new_columns.append(new_data)

    else:
        if isinstance(data['CompanyDetails'],str):
                new_data = {
                                'Name': data['Name'],
                                'DOB': data['DOB'],
                                'Gender': data['Gender'],
                                'FatherName': data['FatherName'],
                                'CitizenshipDetails': data['CitizenshipDetails'],
                                'PassportDetails': data['PassportDetails'],
                                'DrivingLicenseDetails': data['DrivingLicenseDetails'],
                                'VoterIDDetails': data['VoterIDDetails'],
                                'PANDetails': data['PANDetails'],
                                'CompanyDetails': data['CompanyDetails'],
                                'IndianEmbassyDetails': data['IndianEmbassyDetails'],
                                'BlackLists': data['BlackLists']
                            }
                new_columns.append(new_data)
        elif isinstance(data['CompanyDetails'],list):
            for company_element in data['CompanyDetails']:
                    new_data = {
                        'Name': data['Name'],
                        'DOB': data['DOB'],
                        'Gender': data['Gender'],
                        'FatherName': data['FatherName'],
                        'CitizenshipDetails': data['CitizenshipDetails'],
                        'PassportDetails': data['PassportDetails'],
                        'DrivingLicenseDetails': data['DrivingLicenseDetails'],
                        'VoterIDDetails': data['VoterIDDetails'],
                        'PANDetails': data['PANDetails'],
                        'CompanyDetails': company_element,
                        'IndianEmbassyDetails': data['IndianEmbassyDetails'],
                        'BlackLists': data['BlackLists']
                    }
                    new_columns.append(new_data)

        # print(new_columns)
    # if isinstance(data[])

new_df = pd.DataFrame(new_columns)
# new_df

In [4]:
new_df.to_excel('instit.xlsx',index=False)

In [5]:
new_df.head(5)

,Name,DOB,Gender,FatherName,CitizenshipDetails,PassportDetails,DrivingLicenseDetails,VoterIDDetails,PANDetails,CompanyDetails,IndianEmbassyDetails,BlackLists
0,AALI SUB REMIT SEWA PVT. LTD.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,606804729|2075-12-12|Kathmandu,211424/075/076|NULL|Office Of Company Registrar,NaN,Bank and Financial Institutions|32165|2078-11-...
1,RECHAL ELECTRIC CENTRE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,602297651|2071-04-22|Banke,23554-071/072|NULL|Department of commerce and ...,NaN,Bank and Financial Institutions|32166|2078-11-...
2,TIRUPATI FOOT WEAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,602164991|2071-02-22|Sunasari,13105/074/075|NULL|Office of Cottage and Small...,NaN,Bank and Financial Institutions|32809|2078-11-...
3,SUPA DEURALI ALMUNIUM AND STEEL UDHYOG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,611769888|2076-05-18|Rupandehi,19006/076/77|NULL|Office of Cottage and Small ...,NaN,Bank and Financial Institutions|33066|2078-11-...
4,MAHALAXMI DEVI REFRIGERATION WORKSHOP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615455828|2077-11-26|Bhaktapur,63|NULL|District Development Committee,NaN,Bank and Financial Institutions|32147|2078-11-...
